In [237]:
import mysql.connector
from pprint import pprint
import spacy
nlp = spacy.load('en_core_web_lg')

mydb = mysql.connector.connect(
    host="35.226.133.103",
    user="root",
    passwd="keonp2",
    db="project_db"
)

cur = mydb.cursor()
courseNo = 126
courseName = 'Software Design Studio'

In [242]:
def get_prof_rankings_for_course(cur, courseNo, courseName):
    
    def normalize_score(min_s, max_s, score):
        return (score - min_s) / (max_s - min_s)
    
    # 1. get course and its text data
    q = '''
        SELECT courseDesc
        FROM csCourse
        WHERE courseNo = %s AND courseName = %s;
    '''
    
    cur.execute(q, (courseNo, courseName))
    res = cur.fetchone()
    courseDesc = res[0]
    
    course_text = courseName + ", " + courseDesc
    course_text_nlp = nlp(course_text)
    
    # 2. get all instructors' research interests 
    q = '''
        SELECT instructorId, instructorName, researchInterests
        FROM csInstructor;
    '''
    
    cur.execute(q)
    res = cur.fetchall()
    
    prof_research_dict = {}
    prof_id_dict = {}
    for r in res:
        prof_research_dict[r[0]] = nlp(r[2])
        prof_id_dict[r[0]] = str(r[1])
        
    # 3. get average GPA for instructors who have taught the course
    q = '''
        SELECT instructorId, (((SUM(aPlus) * 4) + (SUM(a) * 4) + (SUM(aMinus) * 3.67) + (SUM(bPlus) * 3.33) + (SUM(b) * 3) + (SUM(bMinus) * 2.67) + (SUM(cPlus) * 2.33) + (SUM(c) * 2) + (SUM(cMinus) * 1.67) + (SUM(dPlus) * 1.33) + (SUM(d) * 1) + (SUM(dMinus) * 0.67) + (SUM(f) * 0)) / (SUM(aPlus) + SUM(a) + SUM(aMinus) + SUM(bPlus) + SUM(b) + SUM(bMinus) + SUM(cPlus) + SUM(c) + SUM(cMinus) + SUM(dPlus) + SUM(d) + SUM(dMinus) + SUM(f))) as averageGPA
        FROM csGrade LEFT JOIN csInstructor ON csInstructor.instructorId = csGrade.primaryInstructor
        WHERE csGrade.courseNo = %s AND csGrade.courseName = %s
        GROUP BY csInstructor.instructorId;
    '''
    
    cur.execute(q, (courseNo, courseName))
    res = cur.fetchall()
    
    prof_avg_dict = {}
    for r in res:
        prof_avg_dict[r[0]] = float(r[1])
    base_score = float(min([r[1] for r in res]))
    
    sim_score_weight = 0.1
    
    max_s = 0
    min_s = 0
    prof_total_scores = []
    all_scores = []
    
    for prof in prof_research_dict:
        sim_score = 0
        if prof_research_dict[prof].vector_norm:
            sim_score = float(course_text_nlp.similarity(prof_research_dict[prof]))

        gpa_score = base_score
        if prof in prof_avg_dict:
            gpa_score = prof_avg_dict[prof]
            
        total_score = sim_score * sim_score_weight + gpa_score
            
        prof_data = { 
            "instructorId": prof, 
            "instructorName": prof_id_dict[prof], 
            "researchInterests": prof_research_dict[prof].text, 
            "score": total_score
        }

        all_scores.append(total_score)
        prof_total_scores.append(prof_data)
        
    max_s = max(all_scores)
    min_s = min(all_scores)
    for prof_data in prof_total_scores:
        prof_data["score"] = normalize_score(min_s, max_s, prof_data["score"])
    sorted_prof_scores = sorted(prof_total_scores, key= lambda p : p["score"], reverse=True)
    return sorted_prof_scores[:5]

In [243]:
def get_course_rankings_for_prof(cur, instructorId):
    
    def normalize_score(min_s, max_s, score):
        return (score - min_s) / (max_s - min_s)
    
    # 1. get prof and his/her research interests
    q = '''
        SELECT instructorId, instructorName, researchInterests
        FROM csInstructor
        WHERE instructorId = %s;
    '''
    
    cur.execute(q, (instructorId,))
    instructorId, instructorName, researchInterests = cur.fetchone()
    instructor_text_nlp = nlp(researchInterests)
    
    if not instructor_text_nlp.vector_norm:
        return [] # cant match prof to courses if prof research interests is empty
    
    # 2. get courses and their text data
    q = '''
        SELECT courseNo, courseName, courseDesc
        FROM csCourse;
    '''
    
    cur.execute(q)
    res = cur.fetchall()
    
    course_desc_dict = {}
    for r in res:
        course_desc_dict[(r[0], r[1])] = nlp(r[2])
    
    course_total_scores = []
    for course in course_desc_dict:
        courseNo, courseName = course
        
        score = 0
        if course_desc_dict[course].vector_norm:
            score = instructor_text_nlp.similarity(course_desc_dict[course])
            
        course_data = {
            "courseNo": courseNo,
            "courseName": courseName,
            "courseDesc": course_desc_dict[course].text,
            "score": score
        }
        
        course_total_scores.append(course_data)
        
    sorted_course_scores = sorted(course_total_scores, key = lambda c : c["score"], reverse=True)
    return sorted_course_scores[:10]

In [244]:
# for a given course, get and rank instructors for that course (get top 5)
results = get_prof_rankings_for_course(cur, courseNo, courseName)
pprint(results)

[{'instructorId': 115,
  'instructorName': 'Craig Zilles',
  'researchInterests': 'Learning Analytics, Pedagogy, Computer-Based Testing, '
                       'Assessment, Asynchronous Exams, Item Generation, '
                       'Concept Inventories, Plagiarism Detection',
  'score': 1.0},
 {'instructorId': 76,
  'instructorName': 'Graham Evans',
  'researchInterests': '',
  'score': 0.8809724989921174},
 {'instructorId': 152,
  'instructorName': 'David Padua',
  'researchInterests': 'Compiler Techniques for Parallel Computing, Compiler '
                       'Evaluation and Testing, Autotuning Strategies and '
                       'Systems, Program Analysis, Transformation, and '
                       'Optimization\xa0',
  'score': 0.4487579558835001},
 {'instructorId': 60,
  'instructorName': 'Matthew Caesar',
  'researchInterests': 'Network Verification, Software Resilience, Model '
                       'Checking, Design, Analysis, and Verification of '
              

In [241]:
instructorId = 20
results = get_course_rankings_for_prof(cur, instructorId)
pprint(results)

[{'courseDesc': 'Fundamental principles of computer and communications '
                'security and information assurance: ethics, privacy, notions '
                'of threat, vulnerabilities, and risk in systems, information '
                'warfare, malicious software, data secrecy and integrity '
                'issues, network security, trusted computing, mandatory and '
                'discretionary access controls, certification and '
                'accreditation of systems against security standards. Security '
                'mechanisms: authentication, auditing, intrusion detection, '
                'access control, cryptography, security protocols, key '
                'distribution. ',
  'courseName': 'Computer Security I',
  'courseNo': 461,
  'score': 0.9258387932675674},
 {'courseDesc': 'Program security, trusted base, privacy, anonymity, '
                'non-interference, information flow, confinement, advanced '
                'auditing, forensics, intr